# Capitolo 7: Creare applicazioni di chat
## Guida rapida all’API Github Models

Questo notebook è stato adattato dal [Repository di esempi Azure OpenAI](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst), che include notebook per accedere ai servizi [Azure OpenAI](notebook-azure-openai.ipynb).


# Panoramica  
"I grandi modelli linguistici sono funzioni che associano testo a testo. Dato un input di testo, un grande modello linguistico cerca di prevedere quale testo verrà dopo"(1). Questo notebook "quickstart" introdurrà gli utenti ai concetti principali degli LLM, ai requisiti fondamentali dei pacchetti per iniziare con AML, a una breve introduzione alla progettazione dei prompt e a diversi brevi esempi di casi d’uso differenti.


## Indice  

[Panoramica](../../../../07-building-chat-applications/python)  
[Come utilizzare il servizio OpenAI](../../../../07-building-chat-applications/python)  
[1. Creare il tuo servizio OpenAI](../../../../07-building-chat-applications/python)  
[2. Installazione](../../../../07-building-chat-applications/python)    
[3. Credenziali](../../../../07-building-chat-applications/python)  

[Casi d’uso](../../../../07-building-chat-applications/python)    
[1. Riassumere un testo](../../../../07-building-chat-applications/python)  
[2. Classificare un testo](../../../../07-building-chat-applications/python)  
[3. Generare nuovi nomi di prodotto](../../../../07-building-chat-applications/python)  
[4. Ottimizzare un classificatore](../../../../07-building-chat-applications/python)  

[Riferimenti](../../../../07-building-chat-applications/python)


### Crea il tuo primo prompt  
Questo breve esercizio ti darà un'introduzione di base su come inviare prompt a un modello in Github Models per un compito semplice come la "sintesi".


**Passaggi**:  
1. Installa la libreria `azure-ai-inference` nel tuo ambiente python, se non l'hai già fatto.  
2. Carica le librerie di supporto standard e configura le credenziali per Github Models.  
3. Scegli un modello per il tuo compito  
4. Crea un prompt semplice per il modello  
5. Invia la tua richiesta all'API del modello!


### 1. Installa `azure-ai-inference`


In [ ]:
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

### 3. Trovare il modello giusto  
I modelli GPT-3.5-turbo o GPT-4 sono in grado di comprendere e generare linguaggio naturale.


In [ ]:
# Select the General Purpose curie model for text
model_name = "gpt-4o"

## 4. Progettazione dei prompt  

"La magia dei modelli linguistici di grandi dimensioni sta nel fatto che, essendo addestrati a minimizzare l’errore di previsione su enormi quantità di testo, finiscono per apprendere concetti utili per queste previsioni. Ad esempio, imparano concetti come"(1):

* come si scrive correttamente
* come funziona la grammatica
* come parafrasare
* come rispondere alle domande
* come sostenere una conversazione
* come scrivere in molte lingue
* come programmare
* ecc.

#### Come controllare un modello linguistico di grandi dimensioni  
"Tra tutti gli input di un modello linguistico di grandi dimensioni, il più influente è di gran lunga il prompt testuale(1).

I modelli linguistici di grandi dimensioni possono essere guidati a produrre output in diversi modi:

Istruzione: Dì al modello cosa vuoi
Completamento: Induci il modello a completare l’inizio di ciò che desideri
Dimostrazione: Mostra al modello cosa vuoi, con:
Alcuni esempi nel prompt
Centinaia o migliaia di esempi in un dataset di addestramento per il fine-tuning"



#### Ci sono tre linee guida fondamentali per creare prompt:

**Mostra e spiega**. Fai capire chiaramente cosa vuoi, sia tramite istruzioni, esempi, o una combinazione di entrambi. Se vuoi che il modello ordini una lista di elementi in ordine alfabetico o che classifichi un paragrafo in base al sentimento, mostragli chiaramente cosa ti aspetti.

**Fornisci dati di qualità**. Se stai cercando di costruire un classificatore o di far seguire al modello uno schema, assicurati di fornire abbastanza esempi. Ricontrolla sempre i tuoi esempi — il modello di solito è abbastanza intelligente da ignorare errori di ortografia basilari e darti comunque una risposta, ma potrebbe anche pensare che siano intenzionali e questo può influenzare il risultato.

**Controlla le impostazioni.** I parametri temperature e top_p determinano quanto il modello sia deterministico nel generare una risposta. Se chiedi una risposta dove esiste una sola soluzione corretta, conviene impostarli su valori bassi. Se invece vuoi risposte più varie, puoi alzarli. L’errore più comune con queste impostazioni è pensare che siano controlli di "intelligenza" o "creatività".


Fonte: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Riassumere un testo  
#### Sfida  
Riassumi un testo aggiungendo un 'tl;dr:' alla fine di un passaggio. Nota come il modello sia in grado di svolgere diversi compiti senza istruzioni aggiuntive. Puoi sperimentare con prompt più descrittivi rispetto a tl;dr per modificare il comportamento del modello e personalizzare il riassunto che ricevi(3).  

Studi recenti hanno dimostrato notevoli progressi in molti compiti e benchmark di NLP grazie al pre-addestramento su un ampio corpus di testi, seguito da una fase di fine-tuning su un compito specifico. Sebbene l’architettura sia generalmente indipendente dal compito, questo metodo richiede comunque dataset di fine-tuning specifici composti da migliaia o decine di migliaia di esempi. Al contrario, gli esseri umani sono in grado di affrontare un nuovo compito linguistico con pochi esempi o semplici istruzioni, cosa che i sistemi NLP attuali faticano ancora a fare. Qui mostriamo che aumentare la scala dei modelli linguistici migliora notevolmente le prestazioni few-shot indipendenti dal compito, arrivando talvolta a competere con i precedenti approcci di fine-tuning all’avanguardia. 



Tl;dr


# Esercizi per diversi casi d'uso  
1. Riassumi il testo  
2. Classifica il testo  
3. Genera nuovi nomi di prodotti


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Classifica il testo  
#### Sfida  
Classifica gli elementi nelle categorie fornite al momento dell'inferenza. Nell'esempio seguente, forniamo sia le categorie che il testo da classificare nel prompt (*playground_reference).

Richiesta del cliente: Salve, uno dei tasti della tastiera del mio portatile si è rotto di recente e avrò bisogno di una sostituzione:

Categoria classificata:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Genera Nuovi Nomi di Prodotti
#### Sfida
Crea nomi di prodotti partendo da parole di esempio. Includiamo nel prompt informazioni sul prodotto per cui vogliamo generare i nomi. Forniamo anche un esempio simile per mostrare il modello che desideriamo ricevere. Abbiamo inoltre impostato un valore di temperatura alto per aumentare la casualità e ottenere risposte più innovative.

Descrizione del prodotto: Un frullatore per frappè da casa
Parole chiave: veloce, salutare, compatto.
Nomi di prodotti: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Descrizione del prodotto: Un paio di scarpe che si adattano a qualsiasi misura di piede.
Parole chiave: adattabile, calzata, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# Riferimenti  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [Esempi di OpenAI Studio](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [Best practice per il fine-tuning di GPT-3 per la classificazione del testo](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# Per ulteriore assistenza  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# Collaboratori
* [Chew-Yean Yam](https://www.linkedin.com/in/cyyam/)



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Pur impegnandoci per garantire l’accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa deve essere considerato la fonte autorevole. Per informazioni di carattere critico, si raccomanda una traduzione professionale umana. Non siamo responsabili per eventuali malintesi o interpretazioni errate derivanti dall’uso di questa traduzione.
